<a href="https://colab.research.google.com/github/7201krap/PYTORCH_project/blob/main/all_seeds_3_HL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!wget www.di.ens.fr/~lelarge/MNIST.tar.gz
!tar -zxvf MNIST.tar.gz

root_dir = './'
torchvision.datasets.MNIST(root=root_dir,download=True)

--2021-04-08 16:08:17--  http://www.di.ens.fr/~lelarge/MNIST.tar.gz
Resolving www.di.ens.fr (www.di.ens.fr)... 129.199.99.14
Connecting to www.di.ens.fr (www.di.ens.fr)|129.199.99.14|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.di.ens.fr/~lelarge/MNIST.tar.gz [following]
--2021-04-08 16:08:18--  https://www.di.ens.fr/~lelarge/MNIST.tar.gz
Connecting to www.di.ens.fr (www.di.ens.fr)|129.199.99.14|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/x-gzip]
Saving to: ‘MNIST.tar.gz’

MNIST.tar.gz            [        <=>         ]  33.20M  6.00MB/s    in 16s     

2021-04-08 16:08:35 (2.04 MB/s) - ‘MNIST.tar.gz’ saved [34813078]

MNIST/
MNIST/raw/
MNIST/raw/train-labels-idx1-ubyte
MNIST/raw/t10k-labels-idx1-ubyte.gz
MNIST/raw/t10k-labels-idx1-ubyte
MNIST/raw/t10k-images-idx3-ubyte.gz
MNIST/raw/train-images-idx3-ubyte
MNIST/raw/train-labels-idx1-ubyte.gz
MNIST/raw/t10k-images-idx3-ubyte
MNIST/raw/tra

Dataset MNIST
    Number of datapoints: 60000
    Root location: ./
    Split: Train

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
mnist_trainset = torchvision.datasets.MNIST(root=root_dir, train=True, 
                                download=True, 
                                transform=transforms.Compose([transforms.ToTensor()]))

mnist_testset  = torchvision.datasets.MNIST(root=root_dir, 
                                train=False, 
                                download=True, 
                                transform=transforms.Compose([transforms.ToTensor()]))

train_dataloader = torch.utils.data.DataLoader(mnist_trainset, 
                                               batch_size=50, 
                                               shuffle=True)

test_dataloader  = torch.utils.data.DataLoader(mnist_testset, 
                                               batch_size=50, 
                                               shuffle=False)

In [5]:
# ************* modify this section for later use *************
# Define the model 
class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()

        # modify this section for later use 
        self.linear_1 = torch.nn.Linear(784, 256)
        self.linear_2 = torch.nn.Linear(256, 256)
        self.linear_3 = torch.nn.Linear(256, 256)
        self.linear_4 = torch.nn.Linear(256, 10)
        self.sigmoid12  = torch.nn.Sigmoid()
        self.sigmoid23  = torch.nn.Sigmoid()
        self.sigmoid34  = torch.nn.Sigmoid()

        self.layer_activations = dict()

    def forward(self, x):

        # modify this section for later use 
        x = x.reshape(x.size(0), -1)
        x = self.linear_1(x)
        x = self.sigmoid12(x)
        x = self.linear_2(x)
        x = self.sigmoid23(x)
        x = self.linear_3(x)
        x = self.sigmoid34(x)
        pred = self.linear_4(x)
        return pred
# ************* modify this section for later use *************

In [6]:
def get_activation(model, layer_name):    
    def hook(module, input, output):
        model.layer_activations[layer_name] = output
    return hook

In [7]:
def sparsity_calculator(final_spareness):
    sparseness_list_avg = list()
    sparseness_list_std = list()
    for single_epoch_spareness in final_spareness:

        hidden_layer_activation_list = single_epoch_spareness
        hidden_layer_activation_list = torch.stack(hidden_layer_activation_list)
        layer_activations_list = torch.reshape(hidden_layer_activation_list, (10000, 256))

        layer_activations_list = torch.abs(layer_activations_list)  # modified 
        num_neurons = layer_activations_list.shape[1]
        population_sparseness = (np.sqrt(num_neurons) - (torch.sum(layer_activations_list, dim=1) / torch.sqrt(torch.sum(layer_activations_list ** 2, dim=1)))) / (np.sqrt(num_neurons) - 1)

        mean_sparseness_per_epoch = torch.mean(population_sparseness)
        std_sparseness_per_epoch  = torch.std(population_sparseness)

        sparseness_list_avg.append(mean_sparseness_per_epoch)
        sparseness_list_std.append(std_sparseness_per_epoch)

    return sparseness_list_avg, sparseness_list_std

In [8]:
def selectivity(hidden_layer_each_neuron):
    __selectivity__ = list()
    # I will now try to find the average of each class for each neuron.
    # check out the next cell 
    avg_activations = [dict() for x in range(256)]
    for i, neuron in enumerate(hidden_layer_each_neuron):
        for k, v in neuron.items():
            # v is the list of activations for hidden layer's neuron k 
            avg_activations[i][k] = sum(v) / float(len(v))

    # generate 256 lists to get only values in avg_activations
    only_activation_vals = [list() for x in range(256)]

    # get only values from avg_activations
    for i, avg_activation in enumerate(avg_activations):
        for value in avg_activation.values():
            only_activation_vals[i].append(value)


    for activation_val in only_activation_vals:
        # find u_max 
        u_max = np.max(activation_val)

        # find u_minus_max 
        u_minus_max = (np.sum(activation_val) - u_max) / 9

        # find selectivity 
        selectivity = (u_max - u_minus_max) / (u_max + u_minus_max)

        # append selectivity value to selectivity
        __selectivity__.append(selectivity)

    avg_selectivity = np.average(__selectivity__)
    std_selectivity = np.std(__selectivity__)
                                 
    return avg_selectivity, std_selectivity

In [9]:
# ************* modify this section for later use *************
# add a parameter to the function and calculate avg and std. Do not forget to change division by 2, 3, 4, or 5 
def avg_std_calculator(_hidden_layer_each_neuron_12, _hidden_layer_each_neuron_23, _hidden_layer_each_neuron_34):

    avg_selectivity12, std_selectivity12 = selectivity(_hidden_layer_each_neuron_12)
    avg_selectivity23, std_selectivity23 = selectivity(_hidden_layer_each_neuron_23)
    avg_selectivity34, std_selectivity34 = selectivity(_hidden_layer_each_neuron_34)

    final_selectivity_avg = (avg_selectivity12 + avg_selectivity23 + avg_selectivity34) / 3 
    final_selecvitity_std = (std_selectivity12 + std_selectivity23 + std_selectivity34) / 3 

    return final_selectivity_avg, final_selecvitity_std
# ************* modify this section for later use *************

In [10]:
def model_factory(optimizer_name, seed_num):
    '''
    optimizer_name : choose one of Adagrad, Adadelta, SGD, and Adam 

    '''
    my_model = Model()
    print("my_model:", my_model)
    my_model.to(device)

    # ************* modify this section for later use *************
    # chagen sigmoid34 an 's34'
    my_model.sigmoid12.register_forward_hook(get_activation(my_model, 's12'))
    my_model.sigmoid23.register_forward_hook(get_activation(my_model, 's23'))
    my_model.sigmoid34.register_forward_hook(get_activation(my_model, 's34'))
    # ************* modify this section for later use *************

    if optimizer_name == 'Adadelta':
        my_optimizer = torch.optim.Adadelta(my_model.parameters(), lr=1.0)

    elif optimizer_name == 'Adagrad':
        my_optimizer = torch.optim.Adagrad(my_model.parameters(), lr=0.1)

    elif optimizer_name == 'SGD':
        my_optimizer = torch.optim.SGD(my_model.parameters(), lr=0.1)

    elif optimizer_name == 'Adam':
        my_optimizer = torch.optim.Adam(my_model.parameters(), lr=0.001)

    else:
        print("ERROR")
    
    print("my_optimizer:", my_optimizer)
    test_acc, sparsity_avg, sparsity_std, selectivity_list_avg, selectivity_list_std = sparsity_selectivity_trainer(optimizer=my_optimizer, model=my_model)

    # ************* modify this section for later use *************
    file_saver = open(f"seed_{seed_num}_3HL_{optimizer_name}.txt", "w")
    # ************* modify this section for later use *************
    file_saver.write(str(test_acc)+'\n'+str(sparsity_avg)+'\n'+str(sparsity_std)+'\n'+str(selectivity_list_avg)+'\n'+str(selectivity_list_std)+'\n\n')
    file_saver.close()

    if seed_num == 1:
    # ************* modify this section for later use *************
        if optimizer_name == 'Adadelta':
            !cp seed_1_3HL_Adadelta.txt /content/drive/MyDrive
        
        elif optimizer_name == 'Adagrad':
            !cp seed_1_3HL_Adagrad.txt /content/drive/MyDrive

        elif optimizer_name == 'SGD':
            !cp seed_1_3HL_SGD.txt /content/drive/MyDrive

        elif optimizer_name == 'Adam':
            !cp seed_1_3HL_Adam.txt /content/drive/MyDrive
        # ************* modify this section for later use *************
    
    elif seed_num == 100:
        # ************* modify this section for later use *************
        if optimizer_name == 'Adadelta':
            !cp seed_100_3HL_Adadelta.txt /content/drive/MyDrive
        
        elif optimizer_name == 'Adagrad':
            !cp seed_100_3HL_Adagrad.txt /content/drive/MyDrive

        elif optimizer_name == 'SGD':
            !cp seed_100_3HL_SGD.txt /content/drive/MyDrive

        elif optimizer_name == 'Adam':
            !cp seed_100_3HL_Adam.txt /content/drive/MyDrive
        # ************* modify this section for later use *************

    elif seed_num == 1234:
        # ************* modify this section for later use *************
        if optimizer_name == 'Adadelta':
            !cp seed_1234_3HL_Adadelta.txt /content/drive/MyDrive
        
        elif optimizer_name == 'Adagrad':
            !cp seed_1234_3HL_Adagrad.txt /content/drive/MyDrive

        elif optimizer_name == 'SGD':
            !cp seed_1234_3HL_SGD.txt /content/drive/MyDrive

        elif optimizer_name == 'Adam':
            !cp seed_1234_3HL_Adam.txt /content/drive/MyDrive
        # ************* modify this section for later use *************

    else:
        print("ERROR")

In [11]:
no_epochs = 50
def sparsity_selectivity_trainer(optimizer, model):

    criterion = torch.nn.CrossEntropyLoss()
    train_loss = list()
    test_loss  = list()
    test_acc   = list()

    best_test_loss = 1

    selectivity_avg_list = list()
    selectivity_std_list = list()

    # ************* modify this section for later use *************
    final_spareness_12 = list()
    final_spareness_23 = list()
    final_spareness_34 = list()
    # ************* modify this section for later use *************

    for epoch in range(no_epochs):

        # ************* modify this section for later use *************
        hidden_layer_each_neuron_12 = [{0:[], 1:[], 2:[], 3:[], 4:[], 5:[], 6:[], 7:[], 8:[], 9:[]} for x in range(256)]
        hidden_layer_each_neuron_12 = np.array(hidden_layer_each_neuron_12)

        hidden_layer_each_neuron_23 = [{0:[], 1:[], 2:[], 3:[], 4:[], 5:[], 6:[], 7:[], 8:[], 9:[]} for x in range(256)]
        hidden_layer_each_neuron_23 = np.array(hidden_layer_each_neuron_23)

        hidden_layer_each_neuron_34 = [{0:[], 1:[], 2:[], 3:[], 4:[], 5:[], 6:[], 7:[], 8:[], 9:[]} for x in range(256)]
        hidden_layer_each_neuron_34 = np.array(hidden_layer_each_neuron_34)
        # ************* modify this section for later use *************

        # ************* modify this section for later use *************
        hidden_layer_activation_list_12 = list()
        hidden_layer_activation_list_23 = list()
        hidden_layer_activation_list_34 = list()
        # ************* modify this section for later use *************


        total_train_loss = 0
        total_test_loss = 0

        # training
        # set up training mode 
        model.train()

        for itr, (images, labels) in enumerate(train_dataloader):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()

            pred = model(images)

            loss = criterion(pred, labels)
            total_train_loss += loss.item()

            loss.backward()
            optimizer.step()

        total_train_loss = total_train_loss / (itr + 1)
        train_loss.append(total_train_loss)

        # testing 
        # change to evaluation mode 
        model.eval()
        total = 0
        for itr, (images, labels) in enumerate(test_dataloader):
            images, labels = images.to(device), labels.to(device)

            pred = model(images)

            loss = criterion(pred, labels)
            total_test_loss += loss.item()

            # we now need softmax because we are testing.
            pred = torch.nn.functional.softmax(pred, dim=1)
            for i, p in enumerate(pred):
                if labels[i] == torch.max(p.data, 0)[1]:
                    total = total + 1

            # ***************** sparsity calculation ***************** #
            hidden_layer_activation_list_12.append(model.layer_activations['s12'])
            hidden_layer_activation_list_23.append(model.layer_activations['s23'])
            hidden_layer_activation_list_34.append(model.layer_activations['s34'])

            # ************* modify this section for later use *************
            # Do not forget to change hidden_layer_each_neuron_12 name 
            for activation, label in zip(model.layer_activations['s12'], labels):
                label = label.item()
                with torch.no_grad():
                    activation = activation.numpy()
                for i in range(256):    
                    hidden_layer_each_neuron_12[i][label].append(activation[i])

            for activation, label in zip(model.layer_activations['s23'], labels):
                label = label.item()
                with torch.no_grad():
                    activation = activation.numpy()
                for i in range(256):    
                    hidden_layer_each_neuron_23[i][label].append(activation[i])
            
            for activation, label in zip(model.layer_activations['s34'], labels):
                label = label.item()
                with torch.no_grad():
                    activation = activation.numpy()
                for i in range(256):    
                    hidden_layer_each_neuron_34[i][label].append(activation[i])

        # Add one more parameter 
        selectivity_avg, selecvitity_std = avg_std_calculator(hidden_layer_each_neuron_12, hidden_layer_each_neuron_23, hidden_layer_each_neuron_34)
        # ************* modify this section for later use *************
        
        selectivity_avg_list.append(selectivity_avg)
        selectivity_std_list.append(selecvitity_std)

        # this conains activations for all epochs 
        final_spareness_12.append(hidden_layer_activation_list_12)
        final_spareness_23.append(hidden_layer_activation_list_23)
        final_spareness_34.append(hidden_layer_activation_list_34)
        # ***************** sparsity calculation ***************** #

        # caculate accuracy 
        accuracy = total / len(mnist_testset)

        # append accuracy here
        test_acc.append(accuracy)

        # append test loss here 
        total_test_loss = total_test_loss / (itr + 1)
        test_loss.append(total_test_loss)

        print('\nEpoch: {}/{}, Train Loss: {:.8f}, Test Loss: {:.8f}, Test Accuracy: {:.8f}'.format(epoch + 1, no_epochs, total_train_loss, total_test_loss, accuracy))

    # ***************** sparsity calculation ***************** #
    sparsity_list12_avg, sparsity_list12_std = sparsity_calculator(final_spareness_12)
    sparsity_list23_avg, sparsity_list23_std = sparsity_calculator(final_spareness_23)
    sparsity_list34_avg, sparsity_list34_std = sparsity_calculator(final_spareness_34)

    average_sparsity = list()
    std_sparsity = list()

    for i in range(no_epochs):
        average_sparsity.append( (sparsity_list12_avg[i].item() + sparsity_list23_avg[i].item() + sparsity_list34_avg[i].item()) / 3 )
        std_sparsity.append( (sparsity_list12_std[i].item() + sparsity_list23_std[i].item() + sparsity_list34_std[i].item()) / 3 )
    # ***************** sparsity calculation ***************** #

    return test_acc, average_sparsity, std_sparsity, selectivity_avg_list, selectivity_std_list

In [12]:
# Seed 1

torch.manual_seed(1)
np.random.seed(1)

model_factory('Adadelta',1)
model_factory('Adagrad',1)
model_factory('SGD',1)
model_factory('Adam',1)

my_model: Model(
  (linear_1): Linear(in_features=784, out_features=256, bias=True)
  (linear_2): Linear(in_features=256, out_features=256, bias=True)
  (linear_3): Linear(in_features=256, out_features=256, bias=True)
  (linear_4): Linear(in_features=256, out_features=10, bias=True)
  (sigmoid12): Sigmoid()
  (sigmoid23): Sigmoid()
  (sigmoid34): Sigmoid()
)
my_optimizer: Adadelta (
Parameter Group 0
    eps: 1e-06
    lr: 1.0
    rho: 0.9
    weight_decay: 0
)

Epoch: 1/50, Train Loss: 1.93232611, Test Loss: 0.67482856, Test Accuracy: 0.78650000

Epoch: 2/50, Train Loss: 0.46925898, Test Loss: 0.30330641, Test Accuracy: 0.91060000

Epoch: 3/50, Train Loss: 0.24772453, Test Loss: 0.18281637, Test Accuracy: 0.94580000

Epoch: 4/50, Train Loss: 0.16297404, Test Loss: 0.15234549, Test Accuracy: 0.95460000

Epoch: 5/50, Train Loss: 0.12387935, Test Loss: 0.11584105, Test Accuracy: 0.96370000

Epoch: 6/50, Train Loss: 0.10006041, Test Loss: 0.10570009, Test Accuracy: 0.96780000

Epoch: 7/50

In [13]:
# Seed 100

torch.manual_seed(100)
np.random.seed(100)

model_factory('Adadelta',100)
model_factory('Adagrad',100)
model_factory('SGD',100)
model_factory('Adam',100)

my_model: Model(
  (linear_1): Linear(in_features=784, out_features=256, bias=True)
  (linear_2): Linear(in_features=256, out_features=256, bias=True)
  (linear_3): Linear(in_features=256, out_features=256, bias=True)
  (linear_4): Linear(in_features=256, out_features=10, bias=True)
  (sigmoid12): Sigmoid()
  (sigmoid23): Sigmoid()
  (sigmoid34): Sigmoid()
)
my_optimizer: Adadelta (
Parameter Group 0
    eps: 1e-06
    lr: 1.0
    rho: 0.9
    weight_decay: 0
)

Epoch: 1/50, Train Loss: 2.03231424, Test Loss: 1.05119456, Test Accuracy: 0.58610000

Epoch: 2/50, Train Loss: 0.54536490, Test Loss: 0.31592854, Test Accuracy: 0.90590000

Epoch: 3/50, Train Loss: 0.25831298, Test Loss: 0.19135852, Test Accuracy: 0.94580000

Epoch: 4/50, Train Loss: 0.16669343, Test Loss: 0.16777618, Test Accuracy: 0.94810000

Epoch: 5/50, Train Loss: 0.12780043, Test Loss: 0.12338794, Test Accuracy: 0.96350000

Epoch: 6/50, Train Loss: 0.10347337, Test Loss: 0.11478460, Test Accuracy: 0.96280000

Epoch: 7/50

In [14]:
# Seed 1234

torch.manual_seed(1234)
np.random.seed(1234)

model_factory('Adadelta',1234)
model_factory('Adagrad',1234)
model_factory('SGD',1234)
model_factory('Adam',1234)

my_model: Model(
  (linear_1): Linear(in_features=784, out_features=256, bias=True)
  (linear_2): Linear(in_features=256, out_features=256, bias=True)
  (linear_3): Linear(in_features=256, out_features=256, bias=True)
  (linear_4): Linear(in_features=256, out_features=10, bias=True)
  (sigmoid12): Sigmoid()
  (sigmoid23): Sigmoid()
  (sigmoid34): Sigmoid()
)
my_optimizer: Adadelta (
Parameter Group 0
    eps: 1e-06
    lr: 1.0
    rho: 0.9
    weight_decay: 0
)

Epoch: 1/50, Train Loss: 2.00404090, Test Loss: 0.74558227, Test Accuracy: 0.75990000

Epoch: 2/50, Train Loss: 0.49531609, Test Loss: 0.32430531, Test Accuracy: 0.90510000

Epoch: 3/50, Train Loss: 0.25243455, Test Loss: 0.18649505, Test Accuracy: 0.94610000

Epoch: 4/50, Train Loss: 0.16455400, Test Loss: 0.14236793, Test Accuracy: 0.95980000

Epoch: 5/50, Train Loss: 0.12499757, Test Loss: 0.13524991, Test Accuracy: 0.95950000

Epoch: 6/50, Train Loss: 0.10178312, Test Loss: 0.10451699, Test Accuracy: 0.96890000

Epoch: 7/50